In [2]:
import sys
from os import path
import os
from dotenv import load_dotenv

load_dotenv()
sys.path.append(path.dirname(os.getcwd()))

# Analyze Experiments Result

What do we want to extract from the data?

In [18]:
from project.experiments.models import Experiment
import pandas as pd
import json

experiment_file_name = "/home/leoli/Uni/Polimi/Thesis/master-thesis/notebooks/experiment-1733312932.json"

with open(experiment_file_name, "r") as f:
    experiment_json = json.load(f)

experiment = Experiment(**experiment_json)

video_ids = sorted(list(experiment.predicted_labels_by_model_and_video_id[experiment.models[0]].keys()))
true_labels = pd.read_csv("../data/YouNiCon/conspiracy_label.csv")
true_labels = true_labels[true_labels["video_id"].isin(video_ids)]
true_labels["majority_label"] = true_labels["majority_label"].astype(bool)
conspiracy_videos = true_labels[true_labels["majority_label"] == 1] 
non_conspiracy_videos = true_labels[true_labels["majority_label"] == 0] 

print(true_labels)

         video_id  majority_label
0     -3pY4UfY-Mw            True
2     -8u-hsE87xw            True
3     -9StG9sV4zk            True
4     -K9TdiQPmX4            True
5     -Ph_dpS7uy0            True
...           ...             ...
3155  zt5z02_7ME4           False
3157  zuhZpUX_f_A           False
3158  zwx5NGuu2aQ           False
3159  zzDZVChMM7I           False
3160  zzqJ1F8n_kU            True

[2515 rows x 2 columns]


In [6]:
print(len(experiment.predicted_labels_by_model_and_video_id[experiment.models[0]]))

2515


In [23]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


def generate_confusion_matrix_and_report(true_labels: list[bool], predicted_labels: list[bool]):
    labels = ["non-conspiracy","conspiracy"]
    conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=labels)
    report = classification_report(true_labels, predicted_labels, labels=labels)

    return conf_matrix, report


video_ids_by_model = {}
failed_video_ids_by_model = {}

for model in experiment.models:
    predicted_labels_by_video_id = experiment.predicted_labels_by_model_and_video_id[model]

    video_ids_by_model[model] = [video_id for video_id in video_ids if isinstance(predicted_labels_by_video_id[video_id], dict)]
    failed_video_ids_by_model[model] = [video_id for video_id in video_ids if not isinstance(predicted_labels_by_video_id[video_id], dict)]

    predicted_labels = [predicted_labels_by_video_id[video_id]["is_conspiracy"] for video_id in video_ids_by_model[model]]
    true_labels_for_model = (true_labels[true_labels["video_id"].isin(video_ids_by_model[model])])["majority_label"]

    print(true_labels_for_model)

    conf_matrix, report = generate_confusion_matrix_and_report(true_labels_for_model, predicted_labels)
    
    print(model)
    print("Failed video:", len(failed_video_ids_by_model[model]))
    print(report)
    print()

# disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["0", "1"])
# disp.plot(cmap='Blues', values_format='d')

# plt.title("Confusion Matrix")
# plt.show()



0        True
2        True
3        True
4        True
5        True
        ...  
3155    False
3157    False
3158    False
3159    False
3160     True
Name: majority_label, Length: 2512, dtype: bool


ValueError: At least one label specified must be in y_true